<a href="https://colab.research.google.com/github/alfredcuellv/Proyecto-Final-IA-2023/blob/main/Proyecto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
LIBRERÍAS
"""

from pandas.io.formats.style import Subset
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict, train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, matthews_corrcoef
from sklearn.neural_network import MLPClassifier
from sklearn.utils.validation import column_or_1d

In [ ]:
"""
EXTRACCION Y LIMPIEZA DE DATOS
"""

#Se sabe de antelacion que las etiquetas seran los cultivos y las caracteristicas
#seran las columnas con valores numericos del data set (No todas)

#Extraer los datos
dataCultivos = pd.read_csv('NIRSdata.csv', delimiter= ',', encoding='ISO-8859-1')

#Eliminar las filas donde las etiquetas tengan valores NaN
dataCultivos.dropna(subset=['Cultivo'], inplace=True)

#Eliminar otros valores no deseados en las etiquetas
dataCultivos = dataCultivos[~dataCultivos['Cultivo'].isin(['0', 'Pendiente', 'No Indica'])]

#Definir caracteristicas (Solo se toman las que tienen valores numericos por lo pronto)
numeric_columns = dataCultivos.select_dtypes(include=[float, int]).columns
X = dataCultivos[numeric_columns]

#Eliminar las columnas que sabemos no seran de utilidad
X = X.drop(['Unnamed: 0', '...2.x', '...2.y'],axis = 1)

#Verificar el numero de NaN para cada caracteristica
num_nan_por_columna = X.isna().sum(axis=0)
#Con esto verificamos que caracteristicas tienen muchos NaN para posterior eliminacion
#Tambien verificar que otras caracteristicas tiene algunos NaN llenarlos por metodos de imputacion
#print("La variable X tiene los siguientes valores NaN por columna:")
#print(num_nan_por_columna)

#Para las caracteristicas que suponene un alto numero de valores NaN se eliminan
X = X.drop(['Capacidad de intercambio catiónico (CICA) acetato de amonio','% humedad gravimetrica',
            'Porcentaje de arcilla (% Ar)', 'Porcentaje de arena (% A)', 'Porcentaje de limo (% L)'], axis = 1)

#Mostrar los nombres de los cultivos sin repetir
#print(dataCultivos['Cultivo'].unique())

#Mapeo de etiquetas
#Pasar de letras a numeros las etiquetas
cultivos = dataCultivos['Cultivo'].unique()
#print(len(cultivos))
dict_cultivos = {etiqueta: valor for valor, etiqueta in enumerate(cultivos)}

#Definir etiquetas(Aun no esta limpia del todo)
Y = dataCultivos['Cultivo'].map(dict_cultivos)

#Para los que tienen pocos valores NaN, se pueden deducir por otros metodos.
#Se aplicara KNN_Imputer

#Hacer una matriz general de caracteristicas y etiquetas 
#Concatenar Etiquetas con caracteristicas
#print(X.shape)
#print(Y.shape)
Y = Y.to_numpy()
Y = Y.reshape(3176,1)
YX = np.concatenate((Y, X), axis=1)
#print(YX.shape)
cultivos_num = np.arange(0,len(cultivos)) #Creamos un array que tenga los cultivos, nos servira como indexador
columnas = len(cultivos_num) #Esto es para saber cuantas matrices quedaran
matrices = [] #Inicializar la lista de matrices
num_cultivoX = 0 #Esta variable sirve para crear matrices auxiliares que iran llenando la lista
cont = 0
for i in range(0,columnas):
    cultivo = cultivos_num[i]
    for z in range(0,len(YX)):
      if YX[z,0] == cultivo:
        num_cultivoX += 1
    #print("Cantidad de cultivo ", cultivo, "en data set efectivo = ", num_cultivoX)
    matriz_j = np.zeros((num_cultivoX, 252))
    num_cultivoX = 0
    for j in range(0, len(YX)):
        etiqueta = np.array(YX[j, 0])
        caracteristicas = np.array(YX[j, 1:])
        #Aca se hace el llenado de la matriz de una misma etiqueta
        if etiqueta == cultivo:
          etiqueta = etiqueta.reshape(1, 1)
          caracteristicas = caracteristicas.reshape(1,251)
          matriz_j[cont] = np.concatenate((etiqueta,caracteristicas), axis = 1)
          cont += 1
    cont = 0
    matrices.append(matriz_j)

#LLenar valores NaN
from sklearn.impute import KNNImputer
import math 
#Thumb rule para definir los k vecinos, ya que al ser de diferentes tamaños cada matriz, deberia de
#tener tambien diferentes k
def k_ThumbRule(n):
    raiz = math.floor(math.sqrt(n))
    if raiz % 2 == 0:
        raiz -= 1
    return raiz
def matriz_contiene_nan(matriz):
    return np.isnan(matriz).any()
j_elem = 0
incom = np.zeros((3,), dtype=int)
"""
Debido a que hay cultivos que tienen solo una muestra, estos tendran caracteristicas
incompletas, mas adelante se eliminaran estas, pues se consideran outliners al solo
tener una muestra
"""
for i in range(0,len(matrices)):
  #print(len(matrices[i]))
  k = k_ThumbRule(len(matrices[i]))
  imputer = KNNImputer(n_neighbors=k)
  #print(matrices[i].shape)
  matrices[i] = imputer.fit_transform(matrices[i])
  #print(matrices[i].shape)
  #Aca se verifica que cultivos quedaron incompletos y se guarda su indice para posterior eliminacion
  if len(matrices[i][0]) != 252:
    #print("Matriz de cultivo:",i,"con caracteristicas incompletas")
    incom[j_elem] = i
    #print(incom[j_elem])
    j_elem += 1
    #print(matrices[i].shape)
  #if matriz_contiene_nan(matrices[i]):
    #print("La matriz",i, "contiene valores NaN.")
  #else:
    #print("La matriz",i, "no contiene valores NaN.")

#Elminar cultivos que solo tienen una muestra
for i in range(0,j_elem):
  matriz_eliminar = incom[i] - i
  matrices.pop(matriz_eliminar)

#Crear matriz final, ya limpia y definir las etiquetas y caracteristicas limpias
matriz_final = np.vstack(matrices)
#np.random.shuffle(matriz_final)
Y_limpia = matriz_final[:,0]
Y_limpiaT = Y_limpia.T
Y_limpiaT = Y_limpiaT.reshape(3173,1)
X_limpia = matriz_final[:,1:]

print(X_limpia)
print(Y_limpiaT)



<ipython-input-2-9adc8f63ffe1>:9: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,25,26,27,28,29,30,31,32,35,36,37,38,39,42,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  dataCultivos = pd.read_csv('NIRSdata.csv', delimiter= ',', encoding='ISO-8859-1')


[[ 1.29      18.52       0.49      ...  0.5149937  0.521809   0.527157 ]
 [ 1.29      18.52       0.49      ...  0.5277381  0.5347335  0.540491 ]
 [ 1.69      25.43       0.74      ...  0.5714682  0.5784513  0.5842493]
 ...
 [ 1.22      17.18       0.66      ...  0.4490449  0.4571437  0.4630668]
 [ 7.79       2.51       0.15      ...  0.5050989  0.5098143  0.5134151]
 [ 4.31      21.3        0.39      ...  0.4862949  0.4918318  0.4951547]]
[[ 0.]
 [ 0.]
 [ 0.]
 ...
 [95.]
 [95.]
 [95.]]


In [ ]:
"""
REDUCCIÓN DIMENSIONAL POR PCA
"""

X = X_limpia
Y = Y_limpiaT

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Obtener la varianza explicada acumulativa
pca = PCA()
pca.fit(scaled_data)
varianza_explicada_acumulativa = np.cumsum(pca.explained_variance_ratio_)

# Encontrar el mínimo número de componentes que cumplan el criterio del 99% de varianza explicada
n = np.argmax(varianza_explicada_acumulativa >= 0.99) + 1

# Realizar PCA con el número mínimo de componentes seleccionado
#como se redujo un monton las caracteristicas, se lo pueden agregar un par mas a n
n += 3
pca = PCA(n_components=n)
X_transformado = pca.fit_transform(scaled_data)

print("Número de componentes seleccionados:", n)
print("Pesos de PCA:",pca.explained_variance_ratio_)

X = X_transformado
y = Y.reshape(3173,)

Número de componentes seleccionados: 10
Pesos de PCA: [0.79395679 0.14642609 0.02917655 0.00788464 0.00556766 0.00465292
 0.0033867  0.00293992 0.00234339 0.00160493]


Por SVM

In [ ]:
"""
SVM:
Se usara maquinas de soporte vectorial, implementando cross validation para la separación de los datos
"""


#Empezaremos con maquinas de soporte vectorial con diferentes kernel
#Estos kernels deberian ser adecuados tomando en cuenta la dimensionalidad

# División en conjuntos de entrenamiento y prueba
# Probar con varias semillas
import random

def crear_vector_aleatorio(n):
    vector = []
    for _ in range(n):
        valor = random.randint(0, 100)
        vector.append(valor)
    return vector

Nr = 10  #Numero de semillas
r = crear_vector_aleatorio(Nr)

for i in range(0,Nr):
  # Creacion de un objeto KFold para dividir los datos en k pliegues
  kfold = KFold(n_splits=5, shuffle=True, random_state=r[i])
  y_pred = []
  y_true = []
  # Creacion del modelo SVM con kernel RBF
  svm_model = SVC(kernel='rbf')

  #Se realiza cross validation
  for train_index, test_index in kfold.split(X):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      # Entrenamiento del modelo con conjuntos de entrenamiento
      svm_model.fit(X_train, y_train)
      y_pred.extend(svm_model.predict(X_test))
      y_true.extend(y_test)


  #Evaluar con matthews
  mcc = matthews_corrcoef(y_true, y_pred)
  print('Coeficiente de Matthews:', mcc)

  #Evaluar con F1 Score
  f1 = f1_score(y_true, y_pred, average='weighted')
  print("Puntuación F1:", f1)


Coeficiente de Matthews: 0.29158569063822326
Puntuación F1: 0.32576079649628076
Coeficiente de Matthews: 0.2847069615224632
Puntuación F1: 0.31943418628675013
Coeficiente de Matthews: 0.29038644587228496
Puntuación F1: 0.32586602538677883
Coeficiente de Matthews: 0.29755700197362434
Puntuación F1: 0.3307569303791116
Coeficiente de Matthews: 0.2918393957553591
Puntuación F1: 0.32614048136790624
Coeficiente de Matthews: 0.2893959555235474
Puntuación F1: 0.3233646430144131
Coeficiente de Matthews: 0.2924612389565957
Puntuación F1: 0.32506534559521943
Coeficiente de Matthews: 0.29038644587228496
Puntuación F1: 0.32586602538677883
Coeficiente de Matthews: 0.2999186426226635
Puntuación F1: 0.33116364908828166
Coeficiente de Matthews: 0.2925184158215588
Puntuación F1: 0.325902816262563


Por ANN



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

"""
GRIDSEARCH para ANN
Se aplica busqueda por rejilla para obtener el número de capas ocultas de las redes neuronales
"""

# Define los valores para el número de capas ocultas
hidden_layers = [i for i in range(10, 50)]

# Crea el modelo de red neuronal
model = MLPClassifier()

# Crea el diccionario de parámetros para la búsqueda de cuadrícula
param_grid = {'hidden_layer_sizes': hidden_layers}

# Realiza la búsqueda de cuadrícula
y = column_or_1d(y, warn=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Muestra los resultados de la búsqueda de cuadrícula
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochast

In [ ]:
"""
ANN:
Se usara redes neuronales, implementando cross validation para la separación de los datos
y diferentes funciones de activación
"""

def metricas(mlp, X, y, cv):
  # Calcular métricas
  y_pred = cross_val_predict(mlp, X, y, cv=5)
  f1 = f1_score(y, y_pred, average='weighted')
  matthews = matthews_corrcoef(y, y_pred)

  # Imprimir los resultados de la validación cruzada y las métricas
  print("Precisión promedio:", scores.mean())
  print("Precisión por fold:", scores)
  print("F1 score:", f1)
  print("Coeficiente de Matthews:", matthews)

# Crear un clasificador de Red Neuronal Artificial con función de activación tanh
mlp = MLPClassifier(hidden_layer_sizes=(45,), activation='tanh', learning_rate='adaptive', max_iter=3000)

# Realizar validación cruzada (con 5 folds)
y = column_or_1d(y, warn=True)
cv=5
scores = cross_val_score(mlp, X, y, cv=5, scoring='accuracy')
metricas(mlp, X, y, cv)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(


Precisión promedio: 0.5594078342730818
Precisión por fold: [0.57480315 0.55275591 0.5496063  0.56466877 0.55520505]
F1 score: 0.5398711753105824
Coeficiente de Matthews: 0.47560160521954037


In [ ]:
# Crear un clasificador de Red Neuronal Artificial con función de activación relu
mlp = MLPClassifier(hidden_layer_sizes=(25,), activation='relu', max_iter=3000)

# Realizar validación cruzada (con 5 folds)
y = column_or_1d(y, warn=True)
scores = cross_val_score(mlp, X, y, cv=5, scoring='accuracy')
metricas(mlp, X, y, cv)


# Crear un clasificador de Red Neuronal Artificial con función de activación logistic
mlp = MLPClassifier(hidden_layer_sizes=(25,), activation='logistic', max_iter=3000)

# Realizar validación cruzada (con 5 folds)
y = column_or_1d(y, warn=True)
scores = cross_val_score(mlp, X, y, cv=5, scoring='accuracy')
metricas(mlp, X, y, cv)